# Kelly Criterion Calculator

## Purpose
Calculate optimal position sizing using Kelly Criterion based on backtest results.

## Workflow
1. **Configure strategies** in `config/config.yaml`
2. **Run backtest** using `example_backtest.py`
3. **Run this notebook** to calculate Kelly %
4. **Copy Kelly values** back to `config/config.yaml` → `position_sizing.kelly_pct`
5. **Set method** to `kelly` in config
6. **Re-run backtest** with Kelly-based position sizing

## Kelly Formula
```
f* = (p × b - q) / b

Where:
  f* = Fraction of capital to risk (Kelly %)
  p = Win rate
  q = Loss rate (1 - p)
  b = Payoff ratio = Avg Win / Avg Loss
```

**Important:** Use **Half Kelly** (multiply by 0.5) or **Quarter Kelly** (multiply by 0.25)


In [ ]:
import pandas as pd
import numpy as np
import yaml
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

from src.strategies.vertical_spreads import BullPutSpread, BullCallSpread
from src.strategies.calendar_spreads import CallCalendarSpread
from src.backtester.optopsy_wrapper import OptopsyBacktester
from src.data_fetchers.synthetic_generator import load_sample_spy_options_data
from src.data_fetchers.yahoo_options import fetch_spy_data

## 1. Load Config and Data

In [ ]:
# Load configuration
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Strategies configured:")
for strategy_name, strategy_config in config['strategies'].items():
    enabled = "✓" if strategy_config.get('enabled', False) else "✗"
    print(f"  {enabled} {strategy_name}")
    
    # Show entry parameters
    entry = strategy_config.get('entry', {})
    for key, value in entry.items():
        print(f"      {key}: {value}")

In [ ]:
# Load options data
print("Loading synthetic options data...")
options_data = load_sample_spy_options_data()
print(f"✓ Loaded {len(options_data):,} option rows")

# Load underlying data
print("\nLoading SPY price data...")
start_date = options_data['quote_date'].min().strftime('%Y-%m-%d')
end_date = options_data['quote_date'].max().strftime('%Y-%m-%d')
underlying_data = fetch_spy_data(start_date, end_date)
print(f"✓ Loaded {len(underlying_data)} days of price data")
print(f"  Date range: {start_date} to {end_date}")

## 2. Run Backtests for Each Strategy

In [ ]:
# Initialize backtester
backtester = OptopsyBacktester(config)

# Storage for results
all_results = {}

# Get enabled strategies
enabled_strategies = [
    name for name, cfg in config['strategies'].items() 
    if cfg.get('enabled', False)
]

print(f"Running backtests for {len(enabled_strategies)} strategies...\n")

In [ ]:
# Run bull_put_spread
if 'bull_put_spread' in enabled_strategies:
    print("="*70)
    print("BULL PUT SPREAD")
    print("="*70)
    
    strategy = BullPutSpread(config['strategies']['bull_put_spread'])
    results = backtester.run_backtest(
        strategy=strategy,
        options_data=options_data,
        underlying_data=underlying_data
    )
    
    backtester.print_results(results)
    all_results['bull_put_spread'] = results
    print()

In [ ]:
# Run bull_call_spread
if 'bull_call_spread' in enabled_strategies:
    print("="*70)
    print("BULL CALL SPREAD")
    print("="*70)
    
    strategy = BullCallSpread(config['strategies']['bull_call_spread'])
    results = backtester.run_backtest(
        strategy=strategy,
        options_data=options_data,
        underlying_data=underlying_data
    )
    
    backtester.print_results(results)
    all_results['bull_call_spread'] = results
    print()

In [ ]:
# Run call_calendar
if 'call_calendar' in enabled_strategies:
    print("="*70)
    print("CALL CALENDAR SPREAD")
    print("="*70)
    
    strategy = CallCalendarSpread(config['strategies']['call_calendar'])
    results = backtester.run_backtest(
        strategy=strategy,
        options_data=options_data,
        underlying_data=underlying_data
    )
    
    backtester.print_results(results)
    all_results['call_calendar'] = results
    print()

## 3. Calculate Kelly Criterion

In [ ]:
def calculate_kelly(trades_df):
    """
    Calculate Kelly Criterion from trades DataFrame.
    """
    if len(trades_df) == 0:
        return None
    
    # Filter winning and losing trades
    winning_trades = trades_df[trades_df['net_pnl'] > 0]
    losing_trades = trades_df[trades_df['net_pnl'] < 0]
    
    # Calculate metrics
    total_trades = len(trades_df)
    win_rate = len(winning_trades) / total_trades if total_trades > 0 else 0
    avg_win = winning_trades['net_pnl'].mean() if len(winning_trades) > 0 else 0
    avg_loss = abs(losing_trades['net_pnl'].mean()) if len(losing_trades) > 0 else 0
    
    # Calculate Kelly
    loss_rate = 1 - win_rate
    payoff_ratio = avg_win / avg_loss if avg_loss > 0 else 0
    
    if payoff_ratio > 0:
        kelly_pct = (win_rate * payoff_ratio - loss_rate) / payoff_ratio
    else:
        kelly_pct = 0
    
    return {
        'trades': total_trades,
        'win_rate': win_rate,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'payoff_ratio': payoff_ratio,
        'full_kelly': kelly_pct,
        'half_kelly': kelly_pct * 0.5,
        'quarter_kelly': kelly_pct * 0.25
    }

# Calculate Kelly for each strategy
kelly_results = {}

for strategy_name, results in all_results.items():
    kelly = calculate_kelly(results['trades'])
    if kelly:
        kelly_results[strategy_name] = kelly

## 4. Kelly Results Summary

In [ ]:
print("\n" + "="*80)
print("KELLY CRITERION RESULTS")
print("="*80)

for strategy_name, kelly in kelly_results.items():
    print(f"\n{strategy_name.upper().replace('_', ' ')}:")
    print(f"  Total Trades:    {kelly['trades']}")
    print(f"  Win Rate:        {kelly['win_rate']*100:.2f}%")
    print(f"  Average Win:     ${kelly['avg_win']:.2f}")
    print(f"  Average Loss:    ${kelly['avg_loss']:.2f}")
    print(f"  Payoff Ratio:    {kelly['payoff_ratio']:.3f}")
    print(f"")
    print(f"  Full Kelly:      {kelly['full_kelly']*100:.2f}%")
    print(f"  Half Kelly:      {kelly['half_kelly']*100:.2f}% ← RECOMMENDED")
    print(f"  Quarter Kelly:   {kelly['quarter_kelly']*100:.2f}% ← CONSERVATIVE")

print("\n" + "="*80)

## 5. Config Update Instructions

Copy these values to your `config/config.yaml` file:

In [ ]:
print("\nCOPY TO config/config.yaml:")
print("\n" + "-"*80)
print("position_sizing:")
print("  method: 'kelly'  # Change from 'fixed_risk'")
print("  kelly_pct:")

for strategy_name, kelly in kelly_results.items():
    # Use Half Kelly by default
    recommended_kelly = kelly['half_kelly']
    print(f"    {strategy_name}: {recommended_kelly:.4f}  # Half Kelly")

print("-"*80)

print("\n💡 TIP: These are Half Kelly values (50% of Full Kelly)")
print("   If results are too volatile, use Quarter Kelly (divide by 2)")
print("   If you want more aggressive, use Full Kelly (multiply by 2)")

## 6. Detailed Comparison Table

In [ ]:
# Create comparison DataFrame
comparison_data = []

for strategy_name, kelly in kelly_results.items():
    comparison_data.append({
        'Strategy': strategy_name.replace('_', ' ').title(),
        'Trades': kelly['trades'],
        'Win Rate %': f"{kelly['win_rate']*100:.1f}%",
        'Avg Win $': f"${kelly['avg_win']:.2f}",
        'Avg Loss $': f"${kelly['avg_loss']:.2f}",
        'Payoff Ratio': f"{kelly['payoff_ratio']:.3f}",
        'Full Kelly %': f"{kelly['full_kelly']*100:.2f}%",
        'Half Kelly %': f"{kelly['half_kelly']*100:.2f}%",
        'Quarter Kelly %': f"{kelly['quarter_kelly']*100:.2f}%"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nSTRATEGY COMPARISON:")
print(comparison_df.to_string(index=False))

## Next Steps

1. **Review the Kelly %** values above
2. **Copy the values** to `config/config.yaml` → `position_sizing.kelly_pct`
3. **Set method** to `kelly` in position_sizing
4. **Re-run** `python example_backtest.py` to test Kelly-based sizing
5. **Compare results** with fixed-risk sizing

**Important Notes:**
- Half Kelly is recommended (good balance of growth vs. volatility)
- Quarter Kelly is more conservative (lower drawdowns)
- Full Kelly is too aggressive for most traders
- Recalibrate every 3-6 months as market conditions change